In [2]:
import pandas as pd
import numpy as np
# import local funcionts
from functions import preprocessing_data, vip_criteria, last_time_vip, last_month_status

In [3]:
import json
with open("pur_list.json", "r") as fp:
     pur_list = json.load(fp)

In [7]:
df = pd.DataFrame(pur_list)

# change column format do date
df.date = df.date.astype('datetime64[ns]')

# generate date_month column
#df['date_month'] = df.date.dt.to_period('M')
df['date_month'] = df.date.to_numpy().astype('datetime64[M]')

df.head(9)

,user,amount,date,date_month
0,9,16.10,2020-12-16,2020-12-01
1,8,42.96,2021-01-21,2021-01-01
2,2,44.15,2020-12-13,2020-12-01
3,1,15.96,2021-01-19,2021-01-01
4,7,47.07,2020-12-20,2020-12-01
5,3,27.49,2021-01-20,2021-01-01
6,8,25.95,2020-12-23,2020-12-01
7,7,22.40,2021-02-13,2021-02-01
8,3,38.50,2021-01-18,2021-01-01


## temporary columns

In [5]:

# purchases per month
df_group = df.groupby(['user','date_month'],as_index=False).agg(monthly_amount=('amount','sum'))

# column amount_100
df_group['amount_100'] = np.where(df_group.monthly_amount>=100,1,0)

# sort df by user & date_month
df_group = df_group.sort_values(by=['user','date_month'],ascending=True)

# check if months are consecutive 
df_group['last_month'] = df_group['date_month'].shift(1)

# date_diff check (only consecutive months)
df_group['diff_months'] = (df_group['date_month'].dt.year - df_group['last_month'].dt.year)*12 + (df_group['date_month'].dt.month - df_group['last_month'].dt.month)

df_group['last_amount_100'] = df_group['amount_100'].shift(1)
df_group['last_user'] = df_group['user'].shift(1)
df_group['last_monthly_amount'] = df_group['monthly_amount'].shift(1) # this is only to be used in the last print of the function

df_group.head(100)

,user,date_month,monthly_amount,amount_100,last_month,diff_months,last_amount_100,last_user,last_monthly_amount
0,1,2020-12-01,99.20,0,NaT,NaN,NaN,NaN,NaN
1,1,2021-01-01,36.86,0,2020-12-01,1.0,0.0,1.0,99.20
2,1,2021-02-01,133.32,1,2021-01-01,1.0,0.0,1.0,36.86
3,1,2021-03-01,121.32,1,2021-02-01,1.0,1.0,1.0,133.32
4,1,2021-05-01,121.32,1,2021-03-01,2.0,1.0,1.0,121.32
5,2,2020-12-01,44.15,0,2021-05-01,-5.0,1.0,1.0,121.32
6,2,2021-01-01,11.19,0,2020-12-01,1.0,0.0,2.0,44.15
7,2,2021-02-01,94.92,0,2021-01-01,1.0,0.0,2.0,11.19
8,3,2020-12-01,40.13,0,2021-02-01,-2.0,0.0,2.0,94.92
9,3,2021-01-01,88.69,0,2020-12-01,1.0,0.0,3.0,40.13


# vip creteria 

In [6]:
# vip creteria 
criteria_user = df_group['user'].shift(1) == df_group['user'] # for the same user (instead of using the columns we use the funcionts)
criteria_date = df_group['diff_months'] == 1 # only last month
criteria_amount = (df_group.last_amount_100 == 1) &(df_group.amount_100 == 1)
df_group['vip'] = np.where(criteria_user & criteria_date & criteria_amount, 1, 0)

df_group.head(100)

,user,date_month,monthly_amount,amount_100,last_month,diff_months,last_amount_100,last_user,last_monthly_amount,vip
0,1,2020-12-01,99.20,0,NaT,NaN,NaN,NaN,NaN,0
1,1,2021-01-01,36.86,0,2020-12-01,1.0,0.0,1.0,99.20,0
2,1,2021-02-01,133.32,1,2021-01-01,1.0,0.0,1.0,36.86,0
3,1,2021-03-01,121.32,1,2021-02-01,1.0,1.0,1.0,133.32,1
4,1,2021-05-01,121.32,1,2021-03-01,2.0,1.0,1.0,121.32,0
5,2,2020-12-01,44.15,0,2021-05-01,-5.0,1.0,1.0,121.32,0
6,2,2021-01-01,11.19,0,2020-12-01,1.0,0.0,2.0,44.15,0
7,2,2021-02-01,94.92,0,2021-01-01,1.0,0.0,2.0,11.19,0
8,3,2020-12-01,40.13,0,2021-02-01,-2.0,0.0,2.0,94.92,0
9,3,2021-01-01,88.69,0,2020-12-01,1.0,0.0,3.0,40.13,0


In [9]:
# last time that a user was vip
df_last_vip_filter = df_group[df_group['vip']==1].groupby('user').agg(max_month=('date_month','max'))
df_vips = df_group[df_group['vip']==1]
df_last_vip = pd.merge(df_vips,
                        df_last_vip_filter,
                        how='inner',
                        left_on=['user','date_month'],
                        right_on=['user','max_month'])

In [10]:
df_last_vip.head()

,user,date_month,monthly_amount,amount_100,last_month,diff_months,last_amount_100,last_user,last_monthly_amount,vip,max_month
0,1,2021-03-01,121.32,1,2021-02-01,1.0,1.0,1.0,133.32,1,2021-03-01
1,5,2021-02-01,140.29,1,2021-01-01,1.0,1.0,5.0,132.61,1,2021-02-01


In [12]:
# last month status
df_last_filter = df_group.groupby('user').agg(max_month=('date_month','max'))

df_last_month = pd.merge(df_group,
                        df_last_filter,
                        how='inner',
                        left_on=['user','date_month'],
                        right_on=['user','max_month'])
df_last_month.head(99)

,user,date_month,monthly_amount,amount_100,last_month,diff_months,last_amount_100,last_user,last_monthly_amount,vip,max_month
0,1,2021-05-01,121.32,1,2021-03-01,2.0,1.0,1.0,121.32,0,2021-05-01
1,2,2021-02-01,94.92,0,2021-01-01,1.0,0.0,2.0,11.19,0,2021-02-01
2,3,2021-02-01,15.18,0,2021-01-01,1.0,0.0,3.0,88.69,0,2021-02-01
3,4,2021-02-01,56.45,0,2020-12-01,2.0,0.0,4.0,29.90,0,2021-02-01
4,5,2021-02-01,140.29,1,2021-01-01,1.0,1.0,5.0,132.61,1,2021-02-01
5,6,2021-02-01,25.63,0,2021-01-01,1.0,0.0,6.0,26.57,0,2021-02-01
6,7,2021-02-01,22.40,0,2021-01-01,1.0,0.0,7.0,18.10,0,2021-02-01
7,8,2021-02-01,22.34,0,2021-01-01,1.0,1.0,8.0,137.50,0,2021-02-01
8,9,2021-02-01,25.05,0,2021-01-01,1.0,0.0,9.0,34.43,0,2021-02-01
9,10,2021-02-01,38.84,0,2021-01-01,1.0,0.0,10.0,65.18,0,2021-02-01
